In [50]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# 1. 데이터 가져오기

In [51]:
import pandas as pd
df = pd.read_csv('D:/ai_x/download/shareData/부동산_250213/최종전국평당분양가격(결측치제외).csv',
                encoding='cp949')
df.head()

,지역명,연도,월,평당분양가격
0,서울,2013,12,18189.0
1,부산,2013,12,8111.0
2,대구,2013,12,8080.0
3,인천,2013,12,10204.0
4,광주,2013,12,6098.0


- 지역명2 : 지역명필드는 라벨인코딩하여 추가
- 평당분양가격n : 평당분양가격의 normalization 스케일 조정하여 추가
- 평당분양가격s : 평당분양가격의 standardization 스케일 조정하여 추가
- 지역명2n : 지역명2의 normalization 스케일 조정하여 추가
- 지역명2s : 지역명2의 standardization 스케일 조정하여 추가
- 연도n : 연도의 normalization 스케일 조정하여 추가
- 연도s : 연도의 standardization 스케일 조정하여 추가
- 월n   : 월의 normalization 스케일 조정하여 추가
- 월s   : 월의 standardization 스케일 조정하여 추가

# 2. 지역명의 라벨 인코딩
- 지역명을 라벨인코딩한 지역명2(LabelEncoder 이용)

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['지역명2'] = le.fit_transform(df['지역명'])
# df['지역명2'] = le.fit_transform(df['지역명']).tolist()
import pandas as pd
#df['지역명2'] = pd.Series(le.fit_transform(df['지역명']))
# df['지역명'].value_counts()
# df['지역명2'].value_counts()
df.sample()

,지역명,연도,월,평당분양가격,지역명2
746,경남,2017,8,8299.5,2


# 3. normalization 스케일 조정
- 입력변수(지역명2, 연도, 월)와 타겟변수(평당분양가격) 따로 스케일 조정(MinMaxScaler이용)
- 지역명2n, 연도n, 월n, 평당분양가격n 추가

In [9]:
df[['지역명2','연도','월','평당분양가격']].sample()

,지역명2,연도,월,평당분양가격
2066,0,2024,2,14642.1


In [30]:
# 전체내용 :df
# 독립변수를 넘파이배열로 : df[['지역명2','연도','월']].values
# 종속변수를 넘파이 배열로 :df[['평당분양가격']].values
X_data = df[['지역명2','연도','월']].values
y_data = df[['평당분양가격']].values
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
df[['지역명2n','연도n','월n']] = x_scaler.fit_transform(X_data)
df['평당분양가격n'] = y_scaler.fit_transform(y_data)
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757


# 4. standarization 스케일 조정
- 입력변수와 타겟변수 따로 스케일 조정(StandardScaler이용)
- 지역명2s, 연도s, 월s, 평당분양가격s 필드 추가

In [31]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()
df[['지역명2s','연도s','월s']] = x_scaler.fit_transform(X_data)
df['평당분양가격s'] = y_scaler.fit_transform(y_data)
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,연도s,월s,평당분양가격s
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203


# 5. 지역명을 원핫인코딩

In [32]:
from tensorflow.keras.utils import to_categorical
to_categorical(df['지역명2'])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [44]:
# 지역명, 지역명2
loc_info = df[['지역명','지역명2']].head(17).sort_values(by='지역명2')
loc_column_names = loc_info['지역명'].tolist()
print(loc_column_names)

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']


In [47]:
import numpy as np
df[loc_column_names] = to_categorical(df['지역명2']).astype(np.int16)
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,...,0,1,0,0,0,0,0,0,0,0
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,...,1,0,0,0,0,0,0,0,0,0
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,...,0,0,0,0,0,0,0,0,0,0
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,...,0,0,0,0,1,0,0,0,0,0
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,...,0,0,0,0,0,0,0,0,0,0


In [49]:
pd.options.display.max_columns = 30 # 최대 출력가능한 데이터프레임 열수
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
from tensorflow.keras.utils import to_categorical
# to_categorical(df['지역명'])
import pandas as pd
pd.get_dummies(df['지역명'])

,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2171,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2172,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2173,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2174,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
